# Cleaning 1.2 - Clean unique values in individual dataset

In [1]:
# Indicator for new variables to be cleaned (default is False, set to True if decide to clean more variables)
new_vars_to_clean = True

In [2]:
# Set-up
import pandas as pd
import numpy as np
import sys
from pathlib import Path
CODE_ROOT = Path.cwd().parents[0]
sys.path.append(str(CODE_ROOT))
import config
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment
import os
from create_empty_cleaning_sheet import create_empty_cleaning_sheet

In [3]:
# Load data
labs = pd.read_csv(config.PROCESSED_DATA / "individual_processed_1.csv")

In [4]:
# Load data dictionaries
other_qs_data_dict = pd.read_excel(config.DATA_DICTIONARIES / "data_dictionary.xlsx", sheet_name="Other")

In [5]:
# Create or update the excel workbook with empty sheets
if new_vars_to_clean:

    # File name
    file_name = config.CLEANING_WORKBOOKS / "individual_cleaning_workbook.xlsx"

    # Other questions (one sheet per variable)
    for _, row in other_qs_data_dict.iterrows():   
        sheet_name = row["Variable"]
        comment = row["Comment"] == "Y"
        free_text = row["Free text"] == "Y"
        create_empty_cleaning_sheet(file_name, sheet_name, comment, free_text)

    # Checklist variables (one sheet for all)
    create_empty_cleaning_sheet(file_name, "Checklist", comment=True)

In [6]:
# Run cleaning loop. This will: 
# 1. Merge our data to the cleaning workbook
# 2. Create cleaned variables
# 3. produce a report of the cleaning process
# 4. Update the cleaning workbook with all uncleaned values